In [1]:
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [2]:
# carregando dados
iris = load_iris()
X, Y = shuffle(iris['data'], iris['target'], random_state=1000)
X.shape

(150, 4)

In [4]:
nb0 = GaussianNB()
nb0.fit(X, Y)
print(classification_report(Y, nb0.predict(X), target_names=iris['target_names']))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        50
  versicolor       0.94      0.94      0.94        50
   virginica       0.94      0.94      0.94        50

    accuracy                           0.96       150
   macro avg       0.96      0.96      0.96       150
weighted avg       0.96      0.96      0.96       150



In [5]:
nb_samples = X.shape[0]
nb_labeled = 20

nb_unlabeled = nb_samples - nb_labeled
nb_unlabeled_samples = 2

#Treino inicial com 20 instâncias
X_train = X[:nb_labeled]
Y_train = Y[:nb_labeled]
X_unlabeled = X[nb_labeled:]

In [9]:
X_unlabeled.shape

(130, 4)

In [10]:
while X_train.shape[0] <= nb_samples:
    #cria modelo
    nb = GaussianNB()
    nb.fit(X_train, Y_train)
    
    
    if X_train.shape[0] == nb_samples:
        break
    
    #realiza predição nos dados sem rótulo
    probs = nb.predict_proba(X_unlabeled)
    
    #obtém os top preditos mais confidentes
    top_confidence_idxs = np.argsort(np.max(probs, axis=1)).astype(np.int64)[::-1]
    #seleciona os top nb_unlabeled_samples definido
    selected_idxs = top_confidence_idxs[0:nb_unlabeled_samples]
    #selectiona novos dados para treino 
    X_new_train = X_unlabeled[selected_idxs]
    Y_new_train = nb.predict(X_new_train)
    
    #adiciona novos dados ao treino 
    X_train = np.concatenate((X_train, X_new_train), axis=0)
    Y_train = np.concatenate((Y_train, Y_new_train), axis=0)
    
    #deleta novos dados de treino do conjunto de não rotulados
    X_unlabeled = np.delete(X_unlabeled, selected_idxs, axis=0)


In [11]:
print(classification_report(Y, nb.predict(X), target_names=iris['target_names']))


              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        50
  versicolor       0.95      0.70      0.80        50
   virginica       0.76      0.96      0.85        50

    accuracy                           0.89       150
   macro avg       0.90      0.89      0.88       150
weighted avg       0.90      0.89      0.88       150

